In [1]:
dl20=(at6137 at6923 at6929 at7143 at8285 at9104 at9336 at9503 at9578 at9744 at9762 at9806 at9830 at9847 at9852 at9879 at9883 at9900)

In [ ]:
chmod u+w input/*
cp -L ../assembly-and-annotation/output/04_analyses_with_fixed_at9879/prep_for_decision_making/*.scaffolds-v2.3_liftoff_polished.gff3_polished.fix.gff3 input/

In [ ]:
#cp input/at9879.augustus.gff input/at9879.augustus-liftoff.gff
blsl liftoff-gff3 input/at9879.augustus-liftoff.gff | \
    awk 'BEGIN{IFS=OFS="\t"}$0 ~ /^#/{print $0}$0 !~ /^#/{print $1, "AUGUSTUS", $3, $4, $5, $6, $7, $8, $9}' \
    >input/at9879.augustus.gff
    

In [ ]:
# ideally this would have been fixed upstream but that would need Leon to re-run everything so fuck it.
awk 'BEGIN{IFS=OFS="\t"}{if ($0 ~ /^#/){print $0} else if ($2 == ".") {$2 = "PASA"; print($1, $2, $3, $4, $5, $6, $7, $8, $9)}}' \
    < ../assembly-and-annotation/output/04_analyses_with_fixed_at9879/prep_for_decision_making/at9879.pasa.gff \
    >input/at9879.pasa.fix.gff

In [2]:
for acc in # "${dl20[@]}"
do
    break
    echo
    echo
    echo ${acc}
    grep 'Note=protein_coding_gene' input/${acc}.scaffolds-v2.3_liftoff_polished.gff3_polished.fix.gff3  | grep -Po "\tID=[^;]+" | cut -f 2 -d = >input/${acc}_tairpconly.txt 
    grep -f input/${acc}_tairpconly.txt input/${acc}.scaffolds-v2.3_liftoff_polished.gff3_polished.fix.gff3 > input/${acc}.liftoff.protonly.gff

    if [ "$acc" == "at6137" ]
    then
        files=( input/${acc}.augustus.gff  input/${acc}.liftoff.protonly.gff)
    else
        files=( input/${acc}.augustus.gff  input/${acc}.liftoff.protonly.gff input/${acc}.pasa.fix.gff input/${acc}.tama-v2.fix.gff)
    fi
    
    for file in "${files[@]}"
    do
        (head -n 1 $file | grep -q '^##gff-version') || sed -i -e '1 i\##gff-version 3' $file
    done
    cat "${files[@]}" |
        sortBed -i - |
        mergeBed -i - -s -c 7 -o distinct >input/${acc}_loci.bed
        
    
    grep $acc ../assembly-and-annotation/output/04_analyses_with_fixed_at9879/prep_for_decision_making/All_nlrs_list.txt | awk '{printf("ID=%s$\n",  $3)}' | sort -u -o ${acc}.nlraugids.txt
    grep -f ${acc}.nlraugids.txt input/${acc}.augustus.gff  |
        sortBed -i - |
        intersectBed -wa -a <(awk 'BEGIN{IFS=OFS="\t"}{print $1, $2, $3, ".", "0", $4}' < input/${acc}_loci.bed) -b - -s |
        awk 'BEGIN{IFS=OFS="\t"}{print $1, $2, $3, $6}' > input/${acc}_nlrloci.bed
done

In [4]:
for acc in "${dl20[@]}"
do
    echo -e "\n\n\n${acc}"

    if [ "$acc" == "at6137" ]
    then
        files=( input/${acc}.augustus.gff  input/${acc}.liftoff.protonly.gff)
    else
        files=( input/${acc}.augustus.gff  input/${acc}.liftoff.protonly.gff input/${acc}.pasa.fix.gff)
    fi
    
    python3 -m gffuuu.gffuuuu \
        --decisions output/${acc}_decisions.bed \
        --happy output/${acc}_happy.gff3 \
        --loci input/${acc}_loci.bed \
        "${files[@]}"
done




at6137
loci at input/at6137_loci.bed
N loci: 33514
Added 31972 genes from input/at6137.augustus.gff (0 missing)
Added 26838 genes from input/at6137.liftoff.protonly.gff (0 missing)
Now scan over loci...
Now write GFFs...
DONE! Stats follow

aug_liftoff_cds_agree_use_liftoff	18071
only_aug	7128
aug_liftoff_cds_disagree_use_liftoff	5847
only_liftoff	1972
catchall_multigene_take_liftoff	948



at6923
loci at input/at6923_loci.bed
N loci: 33672
Added 32234 genes from input/at6923.augustus.gff (0 missing)
Added 26966 genes from input/at6923.liftoff.protonly.gff (0 missing)
Added 14640 genes from input/at6923.pasa.fix.gff (0 missing)
Now scan over loci...
Now write GFFs...
DONE! Stats follow

aug_liftoff_pasa_cds_agree_use_pasa	11019
only_aug	6981
aug_liftoff_cds_agree_use_liftoff	6048
aug_liftoff_cds_disagree_use_liftoff	3556
only_liftoff	1926
pasa_liftoff_notaug_cds_agree_use_pasa	1809
catchall_multigene_take_liftoff	1293
liftoff_pasa_aug_disagree_use_liftoff	1257
aug_pasa_cds_agree_use

In [ ]:
sed -i -e 's/\tNone\t/\t.\t/g' output/*.gff3

In [ ]:
#for acc in "${dl20[@]}" at6137
for acc in at9879
do
    echo output/${acc}_nlr_decisions.bed
    wc -l output/${acc}_decisions.bed 
    wc -l  ../assembly-and-annotation/output/04_analyses_with_fixed_at9879/NLRs_list/${acc}.nlrs.genes.augustus.bed 
    intersectBed -wa -a output/${acc}_decisions.bed \
        -b ../assembly-and-annotation/output/04_analyses_with_fixed_at9879/NLRs_list/${acc}.nlrs.genes.augustus.bed \
        -s \
        > output/${acc}_nlr_decisions.bed
done

In [ ]:
ls  